In [1]:
import sys
print(sys.version)
print(sys.executable)

3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
c:\Users\Karl\anaconda3\python.exe


In [7]:
from binance.client import Client  # Correct import
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
import pandas_ta as ta
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
# ... any other libraries you'll use ...

ModuleNotFoundError: No module named 'binance'

### Cell 2: Data Fetching Function throuh (get_kraken_ohlc)

In [5]:
def get_binanceus_ohlc(symbol="BTCUSD", interval="4h", days=30):
    """
    Fetches OHLC data from Binance.US public API.

    :param symbol: Trading pair (e.g., "BTCUSD")
    :param interval: Time interval (e.g., "4h", "1d", "1w")
    :param days: Number of days of data to fetch
    :return: Pandas DataFrame with OHLC data
    """
    client = Client(api_key=None, api_secret=None)  # No keys needed for public API

    # Calculate start time
    start_str = (pd.Timestamp.now() - pd.Timedelta(days=days)).strftime('%Y-%m-%d')

    # Fetch historical klines (candles)
    klines = client.get_historical_klines(symbol, interval, start_str)

    # Convert to DataFrame
    df = pd.DataFrame(klines, columns=[
        'open_time', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])

    # Select and rename columns
    df = df[['open_time', 'open', 'high', 'low', 'close', 'volume']]
    df = df.rename(columns={'open_time': 'time'})

    # Convert to datetime and set index
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    df = df.set_index('time')

    # Convert to numeric types
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[col] = df[col].astype(float)

    return df

### Cell 3: Fetch Data (Run Only Once)


In [ ]:
# Fetch data from Kraken and save to CSV.
# Run this cell ONLY the first time, or when you need to refresh the data.
btc_4h_data = get_kraken_ohlc(days=1825)
btc_4h_data.to_csv("kraken_btc_4h_5years.csv", index=False)
print(btc_4h_data.head())

In [6]:
# Test with 30 days of data
btc_4h_data_test = get_binanceus_ohlc(days=30)
btc_4h_data_test.to_csv("kraken_btc_4h_30days.csv", index=False) #Change file name
print(btc_4h_data_test.head())

NameError: name 'Client' is not defined

### Cell 4: Load Data from CSV (Use for Subsequent Runs)

In [ ]:
# Load data from the CSV file.
# Run this cell instead of Cell 3 after the initial data fetch.
btc_4h_data = pd.read_csv("kraken_btc_4h_5years.csv", parse_dates=['time'], index_col='time')
print(btc_4h_data.head())

### Cell 5: Feature Engineering Function (add_indicators)

In [ ]:
def add_indicators(df):
    # ... (code from feature_engineering.py) ...

### Cell 6: Apply Feature Engineering

In [ ]:
btc_4h_data = add_indicators(btc_4h_data)
print(btc_4h_data.head())  # Check the results

### Cell 7: Clustering Model Function (clustering_model)

In [ ]:
def clustering_model(df_cluster, n_clusters=3):
    # ... (code from models.py) ...

### Cell 8: Perform Clustering and Add to DataFrame

In [ ]:
# Select the features you want to use for clustering
cluster_features = ['RSI', 'MACD', 'volume']  # Example – adjust as needed
df_cluster = btc_4h_data[cluster_features]

clusters = clustering_model(df_cluster)  # Get cluster assignments
btc_4h_data['Cluster'] = clusters  # Add cluster labels to your main DataFrame

print(btc_4h_data.head())  # See the cluster assignments